In [ ]:
!pip install split-folders

In [ ]:
import splitfolders
splitfolders.ratio('../input/plantvillage-dataset/segmented', output="output", seed=1337, ratio=(.8, .1,.1), group_prefix=None)

In [ ]:
from tensorflow.keras import metrics
METRICS = [
      metrics.TruePositives(name='tp'),
      metrics.FalsePositives(name='fp'),
      metrics.TrueNegatives(name='tn'),
      metrics.FalseNegatives(name='fn'), 
      metrics.CategoricalAccuracy(name='accuracy'),
      metrics.Precision(name='precision'),
      metrics.Recall(name='recall'),
      metrics.AUC(name='auc')
]

In [ ]:
BATCH_SIZE = 38
IMG_SIZE = (224, 224)
IMG_SHAPE = IMG_SIZE + (3,)

# ***Using InceptionResNetV2 CNN architecture***

In [ ]:
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#img_width, img_height = 224, 224

train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator=train_datagen.flow_from_directory('./output/train',
                                                 target_size=IMG_SIZE,
                                                 color_mode='rgb',
                                                 batch_size=BATCH_SIZE,
                                                 class_mode='categorical',
                                                 shuffle=True)

val_generator=train_datagen.flow_from_directory('./output/val',
                                                target_size=IMG_SIZE,
                                                 color_mode='rgb',
                                                 batch_size=BATCH_SIZE,
                                                 class_mode='categorical',
                                                 shuffle=True)
test_generator=train_datagen.flow_from_directory('./output/test',
                                                  target_size=IMG_SIZE,
                                                 color_mode='rgb',
                                                 batch_size=BATCH_SIZE,
                                                 class_mode='categorical',
                                                shuffle=True)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D

base_model=tf.keras.applications.InceptionResNetV2(input_shape=IMG_SHAPE, weights='imagenet',include_top=False)

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
x=Dense(1024,activation='relu')(x)
x=Dense(512,activation='relu')(x)
preds=Dense(38,activation='softmax')(x)

In [ ]:
from tensorflow.keras.models import Model

model1=Model(inputs=base_model.input,outputs=preds)

In [ ]:
for layer in model1.layers[:20]:
    layer.trainable=False
for layer in model1.layers[20:]:
    layer.trainable=True

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

cb = [
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=10,
        mode='auto',
        min_delta=0.0002,
        cooldown=5,
        min_lr=10e-8,
        verbose=1,
    )
]


In [ ]:
from tensorflow.keras.optimizers import Adam
model1.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=METRICS)

In [ ]:
#step_size_train=train_generator.n//train_generator.batch_size
model1_history=model1.fit(train_generator,
                    validation_data =val_generator, epochs=50, steps_per_epoch=500, callbacks = cb)
                   #steps_per_epoch=1,
                  

In [ ]:
import matplotlib.pyplot as plt
#acc = model1_history.history['accuracy']
val_acc = model1_history.history['val_accuracy']

#plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.xlim([0,50])
plt.ylim([min(plt.ylim()),1])
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.show()
plt.savefig("InceptionResnetV2_Accuracy.png")

In [ ]:
loss = model1_history.history['loss']
val_loss = model1_history.history['val_loss']

#plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Loss')
plt.xlim([0,50])
plt.ylim([0,plt.ylim()[1]])
plt.title('Validation Loss')
plt.xlabel('Epoch')
plt.show()
plt.savefig("InceptionResnetV2_Loss.png")

In [ ]:
pre = model1_history.history['precision']
val_pre = model1_history.history['val_precision']

#plt.plot(pre, label='Training Precision')
plt.plot(val_pre, label='Validation Precision')
plt.legend(loc='lower right')
plt.ylabel("Precision")
plt.xlim([0,50])
plt.ylim([min(plt.ylim()),1])
plt.title('Validation Precision')
plt.xlabel('Epoch')
plt.show()
plt.savefig("InceptionResnetV2_Precision.png")

In [ ]:
rec = model1_history.history['recall']
val_rec = model1_history.history['val_recall']

#plt.plot(rec, label='Training Recall')
plt.plot(val_rec, label='Validation Recall')
plt.legend(loc='lower right')
plt.ylabel("Recall")
plt.xlim([0,50])
plt.ylim([min(plt.ylim()),1])
plt.title('Validation Recall')
plt.xlabel('Epoch')
plt.show()
plt.savefig("InceptionResnetV2_Recall.png")

In [ ]:
auc = model1_history.history['auc']
val_auc = model1_history.history['val_auc']

#plt.plot(auc, label='Training AUC')
plt.plot(val_auc, label='Validation AUC')
plt.legend(loc='lower right')
plt.ylabel("AUC")
plt.xlim([0,50])
plt.ylim([min(plt.ylim()),1])
plt.title('Validation AUC')
plt.xlabel('Epoch')
plt.show()
plt.savefig("InceptionResnetV2_Auc.png")

In [ ]:
import matplotlib.colors as mcolors
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns
import numpy as np

p = model1.predict(val_generator)
p = np.argmax(p, axis=1)
y_true = val_generator.classes
c = confusion_matrix(p, y_true)

plt.figure(figsize=(48,38))
plt.title('Confusion matrix')
sns.heatmap(c, annot=True)#, color=)
plt.plot()
plt.savefig("InceptionResnetV2_Confusion_matrix.png")
#print(classification_report(p, y_val, digits=3))

# **Prediction**

In [ ]:
Y_pred = model1.predict_generator(test_generator, 5459)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
c = confusion_matrix(test_generator.classes, y_pred)

plt.figure(figsize=(48,38))
plt.title('Confusion matrix')
sns.heatmap(c, annot=True)#, color=)
plt.plot()
plt.savefig("InceptionResnetV2_Confusion_matrix_Prediction.png")
#print('Classification Report')
#target_names = ['Cats', 'Dogs', 'Horse']
#print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

In [ ]:
scores = model1.evaluate_generator(test_generator,5459) #1514 testing images
print(scores)

# **Applying LIME**

In [ ]:


#test_datagen = ImageDataGenerator(rescale=1. / 255)
print('Generating Training Directory Iterator')
test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = datagen.flow_from_directory(
    '../input/cdlime/Dataset_lime',
    target_size=IMG_SIZE,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)


In [ ]:
import numpy as np

idx = 18
X_test, y_test= next(test_generator)
# compute on remaining test data
#pipe_pred_test = model1.predict(X_test)
pipe_pred_prop = model1.predict_on_batch(X_test)
print(np.argmax(y_test[idx]),np.argmax(model1.predict_on_batch(X_test[idx].reshape(1, 224, 224, 3))))

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(X_test[idx], interpolation = 'nearest')
plt.savefig('18_Original Image ')

In [ ]:
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm
explainer = lime_image.LimeImageExplainer(verbose = False)
segmenter = SegmentationAlgorithm('quickshift', max_dist=50)

explanation = explainer.explain_instance(X_test[idx].astype('double'), 
                                         classifier_fn = model1.predict, 
                                         top_labels=6, hide_color=None, num_samples=1000, segmentation_fn=segmenter)

In [ ]:
from skimage.color import label2rgb

temp, mask = explanation.get_image_and_mask(np.argmax(y_test[idx]), positive_only=True, num_features=5, hide_rest=False)
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (8, 4))
ax1.imshow(label2rgb(mask,temp, bg_label = 0), interpolation = 'nearest')
ax1.set_title('Positive Regions for {}'.format(np.argmax(y_test[idx])))
temp, mask = explanation.get_image_and_mask(np.argmax(y_test[idx]), positive_only=False, num_features=5, hide_rest=False)
ax2.imshow(label2rgb(3-mask,temp, bg_label = 0), interpolation = 'nearest')
ax2.set_title('Positive/Negative Regions for {}'.format(np.argmax(y_test[idx])))
plt.savefig('18_quickshift')

In [ ]:
#Select the same class explained on the figures above.
ind =  explanation.top_labels[0]

#Map each explanation weight to the corresponding superpixel
dict_heatmap = dict(explanation.local_exp[ind])
heatmap = np.vectorize(dict_heatmap.get)(explanation.segments) 

#Plot. The visualization makes more sense if a symmetrical colorbar is used.
plt.imshow(heatmap, cmap = 'RdBu', vmin  = -heatmap.max(), vmax = heatmap.max())
plt.colorbar()
plt.savefig('18_quickshift heatmap')

# **Slice**

In [ ]:
segmenter = SegmentationAlgorithm('slic', n_segments=100)

explanation = explainer.explain_instance(X_test[idx].astype('double'), 
                                         classifier_fn = model1.predict, 
                                         top_labels=6, hide_color=None, num_samples=1000, segmentation_fn=segmenter)

In [ ]:
from skimage.color import label2rgb

temp, mask = explanation.get_image_and_mask(np.argmax(y_test[idx]), positive_only=True, num_features=10, hide_rest=False)
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (8, 4))
ax1.imshow(label2rgb(mask,temp, bg_label = 0), interpolation = 'nearest')
ax1.set_title('Positive Regions for {}'.format(np.argmax(y_test[idx])))
temp, mask = explanation.get_image_and_mask(np.argmax(y_test[idx]), positive_only=False, num_features=10, hide_rest=False)
ax2.imshow(label2rgb(3-mask,temp, bg_label = 0), interpolation = 'nearest')
ax2.set_title('Positive/Negative Regions for {}'.format(np.argmax(y_test[idx])))
plt.savefig('13_slic')

In [ ]:
#Select the same class explained on the figures above.
ind =  explanation.top_labels[0]

#Map each explanation weight to the corresponding superpixel
dict_heatmap = dict(explanation.local_exp[ind])
heatmap = np.vectorize(dict_heatmap.get)(explanation.segments) 

#Plot. The visualization makes more sense if a symmetrical colorbar is used.
plt.imshow(heatmap, cmap = 'RdBu', vmin  = -heatmap.max(), vmax = heatmap.max())
plt.colorbar()
plt.savefig('13_slic heatmap')

# **felzenszwalb**

In [ ]:
segmenter = SegmentationAlgorithm('felzenszwalb', min_size=100)

explanation = explainer.explain_instance(X_test[idx].astype('double'), 
                                         classifier_fn = model1.predict_on_batch, 
                                         top_labels=6, hide_color=None, num_samples=1000, segmentation_fn=segmenter)

In [ ]:
from skimage.color import label2rgb

temp, mask = explanation.get_image_and_mask(np.argmax(y_test[idx]), positive_only=True, num_features=5, hide_rest=False)
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (8, 4))
ax1.imshow(label2rgb(mask,temp, bg_label = 0), interpolation = 'nearest')
ax1.set_title('Positive Regions for {}'.format(np.argmax(y_test[idx])))
temp, mask = explanation.get_image_and_mask(np.argmax(y_test[idx]), positive_only=False, num_features=5, hide_rest=False)
ax2.imshow(label2rgb(3-mask,temp, bg_label = 0), interpolation = 'nearest')
ax2.set_title('Positive/Negative Regions for {}'.format(np.argmax(y_test[idx])))

plt.savefig('13_felzenszwalb')

In [ ]:
#Select the same class explained on the figures above.
ind =  explanation.top_labels[0]

#Map each explanation weight to the corresponding superpixel
dict_heatmap = dict(explanation.local_exp[ind])
heatmap = np.vectorize(dict_heatmap.get)(explanation.segments) 

#Plot. The visualization makes more sense if a symmetrical colorbar is used.
plt.imshow(heatmap, cmap = 'RdBu', vmin  = -heatmap.max(), vmax = heatmap.max())
plt.colorbar()
plt.savefig('13_felzenszwalb heatmap')

**Default Lime**

In [ ]:
%%time
explanation = explainer.explain_instance(X_test[idx].astype('double'), 
                                         classifier_fn = model1.predict_on_batch, 
                                         top_labels=6, hide_color=None, num_samples=1000)#, segmentation_fn=segmenter)

In [ ]:
from skimage.color import label2rgb

temp, mask = explanation.get_image_and_mask(np.argmax(y_test[idx]), positive_only=True, num_features=5, hide_rest=False)
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (8, 4))
ax1.imshow(label2rgb(mask,temp, bg_label = 0), interpolation = 'nearest')
ax1.set_title('Positive Regions for {}'.format(np.argmax(y_test[idx])))
temp, mask = explanation.get_image_and_mask(np.argmax(y_test[idx]), positive_only=False, num_features=5, hide_rest=False)
ax2.imshow(label2rgb(3-mask,temp, bg_label = 0), interpolation = 'nearest')
ax2.set_title('Positive/Negative Regions for {}'.format(np.argmax(y_test[idx])))

plt.savefig('13_Default Lime')

In [ ]:
#Select the same class explained on the figures above.
ind =  explanation.top_labels[0]

#Map each explanation weight to the corresponding superpixel
dict_heatmap = dict(explanation.local_exp[ind])
heatmap = np.vectorize(dict_heatmap.get)(explanation.segments) 

#Plot. The visualization makes more sense if a symmetrical colorbar is used.
plt.imshow(heatmap, cmap = 'RdBu', vmin  = -heatmap.max(), vmax = heatmap.max())
plt.colorbar()

plt.savefig('13_Default Lime HeatMap')